# Week 3
## 1. Data AUC.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_auc = pd.read_csv('dataset/AUC.csv')
df_auc.head(15)

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00
5,The Hunger Games: Special Edition,Writtenby:SuzanneCollins,Narratedby:TatianaMaslany,10 hrs and 35 mins,30-10-18,English,5 out of 5 stars72 ratings,656.00
6,Quest for the Diamond Sword,Writtenby:WinterMorgan,Narratedby:LukeDaniels,2 hrs and 23 mins,25-11-14,English,5 out of 5 stars11 ratings,233.00
7,The Dark Prophecy,Writtenby:RickRiordan,Narratedby:RobbieDaymond,12 hrs and 32 mins,02-05-17,English,5 out of 5 stars50 ratings,820.00
8,Merlin Mission Collection,Writtenby:MaryPopeOsborne,Narratedby:MaryPopeOsborne,10 hrs and 56 mins,02-05-17,English,5 out of 5 stars5 ratings,"1,256.00"
9,The Tyrant’s Tomb,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 22 mins,24-09-19,English,5 out of 5 stars58 ratings,820.00


In [3]:
def check_data(df):
    print(f"There are {df.isna().sum().sum()} missing values in the dataset.")
    print(f"There are {df.duplicated().sum()} duplicated values in the dataset.")

check_data(df_auc)

There are 0 missing values in the dataset.
There are 0 duplicated values in the dataset.


**TODO**:
- Fixing & Removing unnecessary format:
    - Removing:
        - "Writtenby:" in `author` column.
        - "Narratedby:" in `narrator` column.
    - Fixing:
        - Add space between two or more word blocks in `author` and `narrator` columns.
- Turn `Time` column into a minute of duration instead
- Convert some column with corrected data type:
    - Turn `Time` column into a minute of duration instead.
    - Turn `price` column into float instead.
- Turn `release-date` into a pandas datetime format.
- Fix `stars` column by:
    - Adding a new column called `rating` containing how many ratings are there.
    - Fix the stars rating format.

1. Fixing & Removing unnecessary format:


In [4]:
# Removing:
# - "Writtenby:" in `author` column
# - "Narratedby:" in `narrator` column

df_auc['author'] = df_auc['author'].str.replace('Writtenby:', '', regex=False)
df_auc['narrator'] = df_auc['narrator'].str.replace('Narratedby:', '', regex=False)

df_auc.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,JeffKinney,DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [5]:
# Fixing:
# - Add space between two or more word blocks in `author` and `narrator` columns
df_auc['author'] = df_auc['author'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)
df_auc['narrator'] = df_auc['narrator'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)

df_auc.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Geronimo Stilton,Bill Lobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Rick Riordan,Robbie Daymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Jeff Kinney,Dan Russell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Rick Riordan,Soneela Nankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,Jesse Bernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [6]:
df_auc = pd.concat([df_auc['name'], 
                            #splits author into author1, author2 and author3.
                            df_auc['author'].str.split(',', expand = True).add_prefix('author'),
                            df_auc.loc[:,['narrator', 'time', 'releasedate', 'language', 'stars', 'price']]], 
                            axis = 1)

# adding "None" to author 1, 2, 3 if there's no author (missing values)
df_auc[['author1', 'author2', 'author3']] = df_auc[['author1', 'author2', 'author3']].fillna('No Author')

df_auc.head()

,name,author0,author1,author2,author3,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,No Author,Bill Lobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Rick Riordan,No Author,No Author,No Author,Robbie Daymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Jeff Kinney,No Author,No Author,No Author,Dan Russell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Rick Riordan,No Author,No Author,No Author,Soneela Nankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,No Author,No Author,No Author,Jesse Bernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [7]:
df_auc.sample(5, random_state=6)

,name,author0,author1,author2,author3,narrator,time,releasedate,language,stars,price
8407,Night of the New Magicians,Mary Pope Osborne,No Author,No Author,No Author,Mary Pope Osborne,1 hr and 17 mins,13-03-06,English,5 out of 5 stars1 rating,301.00
11786,Is Everyone Really Equal?,Özlem Sensoy,Robin Di Angelo,No Author,No Author,Kirsten Potter,10 hrs and 58 mins,30-03-21,English,Not rated yet,703.00
71037,Slayed by Darkness,Alexandra Ivy,No Author,No Author,No Author,Tanya Eby,2 hrs and 41 mins,29-06-21,English,Not rated yet,352.00
1004,The Princess in Black and the Science Fair Scare,Shannon Hale,Dean Hale,No Author,No Author,Julia Whelan,18 mins,25-09-18,English,Not rated yet,352.00
71202,Ravaged,Elisabeth Naughton,No Author,No Author,No Author,Elizabeth Wiley,5 hrs and 1 min,26-01-16,English,Not rated yet,668.00


In [8]:
df_auc.author0.value_counts()

author0
矢島雅弘                   874
Smart Reading          405
中西貴之                   311
div.                   276
William Shakespeare    242
                      ... 
Gillian Riley            1
John Elmore              1
Ralf Herter              1
Ulrike Wolpers           1
Stephen O'Shea           1
Name: count, Length: 44839, dtype: int64

In [9]:
df_auc.author1.value_counts()

author1
No Author             73762
石橋遊                     874
B J                     311
Mark Victor Hansen       65
Yannick Esters           65
                      ...  
Paul Reilly               1
Sean Callahan             1
Doug Fletcher             1
Marylou Tyler             1
Hamish Hamilton           1
Name: count, Length: 9157, dtype: int64

In [10]:
df_auc.author2.value_counts()

author2
No Author                   85135
Twin Sisters                   19
Amy Newmark                    14
Amy Newmark-editor             12
Pauline Verduzier               9
                            ...  
James Wood                      1
Johan Rockström-foreword        1
Jeffrey F.Ma                    1
John P.Kotter                   1
Agneta Cras                     1
Name: count, Length: 1999, dtype: int64

In [11]:
df_auc.author3.value_counts()

author3
No Author    86713
               776
Name: count, dtype: int64

Apparently there's no author in `author3`, let's remove the column


In [12]:
df_auc.drop('author3', axis=1, inplace=True)
df_auc.head(3)

,name,author0,author1,author2,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,Bill Lobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Rick Riordan,No Author,No Author,Robbie Daymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Jeff Kinney,No Author,No Author,Dan Russell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00


2. Turn `Time` column into a minute of duration instead

In [13]:
# Function to convert 'time' to minutes
def time_to_minutes(time_str):
    # Use regular expressions to extract hours and minutes
    hours_match = re.search(r'(\d+) hrs', time_str)
    minutes_match = re.search(r'(\d+) mins', time_str)
    
    hours = int(hours_match.group(1)) if hours_match else 0
    minutes = int(minutes_match.group(1)) if minutes_match else 0
    
    return hours * 60 + minutes

# Apply the function to the 'time' column
df_auc['duration_in_minutes'] = df_auc['time'].apply(time_to_minutes)

df_auc.drop('time', axis=1, inplace=True)

df_auc.head()

,name,author0,author1,author2,narrator,releasedate,language,stars,price,duration_in_minutes
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,Bill Lobely,04-08-08,English,5 out of 5 stars34 ratings,468.00,140
1,The Burning Maze,Rick Riordan,No Author,No Author,Robbie Daymond,01-05-18,English,4.5 out of 5 stars41 ratings,820.00,788
2,The Deep End,Jeff Kinney,No Author,No Author,Dan Russell,06-11-20,English,4.5 out of 5 stars38 ratings,410.00,123
3,Daughter of the Deep,Rick Riordan,No Author,No Author,Soneela Nankani,05-10-21,English,4.5 out of 5 stars12 ratings,615.00,676
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,No Author,No Author,Jesse Bernstein,13-01-10,English,4.5 out of 5 stars181 ratings,820.00,600


3. Convert some column with corrected data type

In [14]:
# releasedate
df_auc['releasedate'] = pd.to_datetime(df_auc['releasedate'], format = '%d-%m-%y')

df_auc.head(2)

,name,author0,author1,author2,narrator,releasedate,language,stars,price,duration_in_minutes
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,Bill Lobely,2008-08-04,English,5 out of 5 stars34 ratings,468.00,140
1,The Burning Maze,Rick Riordan,No Author,No Author,Robbie Daymond,2018-05-01,English,4.5 out of 5 stars41 ratings,820.00,788


In [15]:
df_auc.price.value_counts()

price
586.00      5533
668.00      4262
703.00      3588
836.00      2704
820.00      2458
            ... 
2,631.00       1
2,764.00       1
895.00         1
364.00         1
544.00         1
Name: count, Length: 1011, dtype: int64

In [16]:
print(f'There are: {len(df_auc[df_auc["price"] == "Free"])} free books') # apparently there are 338 free books

There are: 338 free books


In [17]:
# replace Free with 0
df_auc['price'] = df_auc['price'].str.replace('Free', '0')

# removing commas (,) from price
df_auc['price'] = df_auc['price'].str.replace(',', '')

# convert to float
df_auc['price'] = df_auc['price'].astype(float)

In [18]:
df_auc.sample(3)

,name,author0,author1,author2,narrator,releasedate,language,stars,price,duration_in_minutes
20422,Exactly,Simon Winchester,No Author,No Author,Simon Winchester,2018-05-08,English,Not rated yet,492.0,705
31616,The 10-Day Alcohol Detox Plan,Lewis David,No Author,No Author,Jason Zenobia,2020-10-27,English,Not rated yet,501.0,254
12540,"Learn German - Survival Phrases German, Volume...",Innovative Language Learning,No Author,No Author,German Pod101.com,2009-06-17,English,5 out of 5 stars1 rating,305.0,191


In [19]:
df_auc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 87489 non-null  object        
 1   author0              87489 non-null  object        
 2   author1              87489 non-null  object        
 3   author2              87489 non-null  object        
 4   narrator             87489 non-null  object        
 5   releasedate          87489 non-null  datetime64[ns]
 6   language             87489 non-null  object        
 7   stars                87489 non-null  object        
 8   price                87489 non-null  float64       
 9   duration_in_minutes  87489 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 6.7+ MB


FIx `stars` column

In [20]:
df_auc.stars.value_counts()

stars
Not rated yet                    72417
5 out of 5 stars1 rating          3739
4 out of 5 stars1 rating          1254
5 out of 5 stars2 ratings          998
4.5 out of 5 stars2 ratings        661
                                 ...  
3.5 out of 5 stars61 ratings         1
5 out of 5 stars321 ratings          1
5 out of 5 stars93 ratings           1
4.5 out of 5 stars747 ratings        1
4 out of 5 stars76 ratings           1
Name: count, Length: 665, dtype: int64

In [21]:
# turn stars into rating_stars and total_rating
df_auc[['rating_stars_out_of_5', 'total_rating']] = df_auc['stars'].str.split('stars', expand=True)
df_auc['total_rating'] = df_auc['total_rating'].str.replace('out of', '/')


# removing "rating" text from total_rating with emtpy string
df_auc['total_rating'] = df_auc['total_rating'].str.replace(' ratings', '')

df_auc.drop('stars', axis=1, inplace=True)

In [22]:
df_auc.sample(5, random_state=42)

,name,author0,author1,author2,narrator,releasedate,language,price,duration_in_minutes,rating_stars_out_of_5,total_rating
34975,Die Armada der Irren,Jens Fischer Rodrian,No Author,No Author,Allyson Voller,2022-03-28,german,468.0,290,Not rated yet,None
50744,A Man Without Breath,Philip Kerr,No Author,No Author,Jonathan Keeble,2022-03-03,English,873.0,895,Not rated yet,None
22107,Crashing Through,Robert Kurson,No Author,No Author,Doug Ordunio,2007-04-30,English,1005.0,612,5 out of 5,1 rating
7759,Nate the Great and the Earth Day Robot,Andrew Sharmat,No Author,No Author,John Lavelle,2021-05-25,English,100.0,24,Not rated yet,None
43690,Annabel,Kathleen Winter,No Author,No Author,Laurence Bouvard,2011-06-10,English,752.0,704,Not rated yet,None


In [23]:
df_auc.total_rating.value_counts()

total_rating
1 rating    5822
2           2315
3           1207
4            857
5            535
            ... 
5,500          1
536            1
287            1
2,236          1
126            1
Name: count, Length: 384, dtype: int64

In [24]:
check_data(df_auc)

There are 72417 missing values in the dataset.
There are 1 duplicated values in the dataset.


In [25]:
# changing "1 rating" into 1
df_auc['total_rating'] = df_auc['total_rating'].str.replace('1 rating', '1')
df_auc.sample(5, random_state=42)

,name,author0,author1,author2,narrator,releasedate,language,price,duration_in_minutes,rating_stars_out_of_5,total_rating
34975,Die Armada der Irren,Jens Fischer Rodrian,No Author,No Author,Allyson Voller,2022-03-28,german,468.0,290,Not rated yet,None
50744,A Man Without Breath,Philip Kerr,No Author,No Author,Jonathan Keeble,2022-03-03,English,873.0,895,Not rated yet,None
22107,Crashing Through,Robert Kurson,No Author,No Author,Doug Ordunio,2007-04-30,English,1005.0,612,5 out of 5,1
7759,Nate the Great and the Earth Day Robot,Andrew Sharmat,No Author,No Author,John Lavelle,2021-05-25,English,100.0,24,Not rated yet,None
43690,Annabel,Kathleen Winter,No Author,No Author,Laurence Bouvard,2011-06-10,English,752.0,704,Not rated yet,None


In [26]:
df_auc['rating_stars_out_of_5'].value_counts()

rating_stars_out_of_5
Not rated yet    72417
5 out of 5        6761
4.5 out of 5      4046
4 out of 5        2729
3 out of 5         711
3.5 out of 5       413
2 out of 5         182
1 out of 5         175
2.5 out of 5        47
1.5 out of 5         8
Name: count, dtype: int64

In [27]:
# replacing 'Not rated yet' with 0 in `rating_stars_out_of_5`
df_auc['rating_stars_out_of_5'] = df_auc['rating_stars_out_of_5'].str.replace('Not rated yet', '0')

# removing "out of 5" in `rating_stars_out_of_5` and convert to float datatype
df_auc['rating_stars_out_of_5'] = df_auc['rating_stars_out_of_5'].str.replace('out of 5', '').astype('float') 

In [28]:
# removing commas from total_rating
df_auc['total_rating'] = df_auc['total_rating'].str.replace(',', '')

# fill missing value with 0
df_auc['total_rating'] = df_auc['total_rating'].fillna(0)

# convert to int
df_auc['total_rating'] = df_auc['total_rating'].astype('int64')

In [29]:
# checking the missing values again
df_auc.isna().sum()


name                     0
author0                  0
author1                  0
author2                  0
narrator                 0
releasedate              0
language                 0
price                    0
duration_in_minutes      0
rating_stars_out_of_5    0
total_rating             0
dtype: int64

In [30]:
df_auc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   name                   87489 non-null  object        
 1   author0                87489 non-null  object        
 2   author1                87489 non-null  object        
 3   author2                87489 non-null  object        
 4   narrator               87489 non-null  object        
 5   releasedate            87489 non-null  datetime64[ns]
 6   language               87489 non-null  object        
 7   price                  87489 non-null  float64       
 8   duration_in_minutes    87489 non-null  int64         
 9   rating_stars_out_of_5  87489 non-null  float64       
 10  total_rating           87489 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 7.3+ MB


In [31]:
df_auc.head()

,name,author0,author1,author2,narrator,releasedate,language,price,duration_in_minutes,rating_stars_out_of_5,total_rating
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,Bill Lobely,2008-08-04,English,468.0,140,5.0,34
1,The Burning Maze,Rick Riordan,No Author,No Author,Robbie Daymond,2018-05-01,English,820.0,788,4.5,41
2,The Deep End,Jeff Kinney,No Author,No Author,Dan Russell,2020-11-06,English,410.0,123,4.5,38
3,Daughter of the Deep,Rick Riordan,No Author,No Author,Soneela Nankani,2021-10-05,English,615.0,676,4.5,12
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,No Author,No Author,Jesse Bernstein,2010-01-13,English,820.0,600,4.5,181


In [32]:
check_data(df_auc)

There are 0 missing values in the dataset.
There are 1 duplicated values in the dataset.


In [33]:
df_auc.isna().sum()

name                     0
author0                  0
author1                  0
author2                  0
narrator                 0
releasedate              0
language                 0
price                    0
duration_in_minutes      0
rating_stars_out_of_5    0
total_rating             0
dtype: int64

In [34]:
# find  the duplicated rows
df_auc[df_auc.duplicated(keep=False)]

,name,author0,author1,author2,narrator,releasedate,language,price,duration_in_minutes,rating_stars_out_of_5,total_rating
14322,London,Peter Ackroyd,No Author,No Author,Simon Callow,2005-11-02,English,410.0,180,0.0,0
39603,London,Peter Ackroyd,No Author,No Author,Simon Callow,2005-11-02,English,410.0,180,0.0,0


In [35]:
df_auc = df_auc.drop_duplicates()
check_data(df_auc)

There are 0 missing values in the dataset.
There are 0 duplicated values in the dataset.


In [36]:
df_auc.head()

,name,author0,author1,author2,narrator,releasedate,language,price,duration_in_minutes,rating_stars_out_of_5,total_rating
0,Geronimo Stilton #11 & #12,Geronimo Stilton,No Author,No Author,Bill Lobely,2008-08-04,English,468.0,140,5.0,34
1,The Burning Maze,Rick Riordan,No Author,No Author,Robbie Daymond,2018-05-01,English,820.0,788,4.5,41
2,The Deep End,Jeff Kinney,No Author,No Author,Dan Russell,2020-11-06,English,410.0,123,4.5,38
3,Daughter of the Deep,Rick Riordan,No Author,No Author,Soneela Nankani,2021-10-05,English,615.0,676,4.5,12
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,No Author,No Author,Jesse Bernstein,2010-01-13,English,820.0,600,4.5,181


In [37]:
df_auc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87488 entries, 0 to 87488
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   name                   87488 non-null  object        
 1   author0                87488 non-null  object        
 2   author1                87488 non-null  object        
 3   author2                87488 non-null  object        
 4   narrator               87488 non-null  object        
 5   releasedate            87488 non-null  datetime64[ns]
 6   language               87488 non-null  object        
 7   price                  87488 non-null  float64       
 8   duration_in_minutes    87488 non-null  int64         
 9   rating_stars_out_of_5  87488 non-null  float64       
 10  total_rating           87488 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 8.0+ MB


TODO NEXT:
- Add hour and minute from duration in minutes

In [38]:
# submit our clean dataset
df_auc.to_csv('clean_dataset/clean_auc.csv', index=False)

In [39]:
df_auc.isna().sum()

name                     0
author0                  0
author1                  0
author2                  0
narrator                 0
releasedate              0
language                 0
price                    0
duration_in_minutes      0
rating_stars_out_of_5    0
total_rating             0
dtype: int64